在整个机器学习过程中，除了训练模型外，应该就属数据预处理过程消耗的精力最多，数据预处理过程需要完成的任务包括数据读取、过滤、转换等等。为了将用户从繁杂的预处理操作中解放处理，更多地将精力放在算法建模上，TensorFlow中提供了data模块，这一模块以多种方式提供了数据读取、数据处理、数据保存等功能。本文重点是data模块中的Dataset对象。

# 1 创建

对于创建Dataset对象，官方文档中总结为两种方式，我将这两种方式细化后总结为4中方式：  

**（1）通过Dataset中的range()方法创建包含一定序列的Dataset对象。**
- **[range()](https://tensorflow.google.cn/api_docs/python/tf/data/Dataset#range)**

range()方法是Dataset内部定义的一个的静态方法，可以直接通过类名调用。另外，Dataset中的range()方法与Python本身内置的range()方法接受参数形式是一致的，可以接受range(begin)、range(begin, end)、range（begin, end, step）等多种方式传参。

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
dataset1 = tf.data.Dataset.range(5)
type(dataset1)

tensorflow.python.data.ops.range_op._RangeDataset

注：RangeDataset是Dataset的一个子类。
Dataset对象属于可迭代对象， 可通过循环进行遍历：

In [3]:
for i in dataset1:
    print(i)
    print(i.numpy())

tf.Tensor(0, shape=(), dtype=int64)
0
tf.Tensor(1, shape=(), dtype=int64)
1
tf.Tensor(2, shape=(), dtype=int64)
2
tf.Tensor(3, shape=(), dtype=int64)
3
tf.Tensor(4, shape=(), dtype=int64)
4


可以看到，range()方法创建的Dataset对象内部每一个元素都以Tensor对象的形式存在，可以通过numpy()方法访问真实值。 
- **[from_generator()](https://tensorflow.google.cn/guide/data#consuming_python_generators)**

如果你觉得range()方法不够灵活，功能不够强大，那么你可以尝试使用from_generator()方法。from_generator()方法接收一个可调用的生成器函数最为参数，在遍历from_generator()方法返回的Dataset对象过程中不断生成新的数据，减少内存占用，这在大数据集中很有用。

In [4]:
def count(stop):
  i = 0
  while i<stop:
    print('第%s次调用……'%i)
    yield i
    i += 1

In [5]:
dataset2 = tf.data.Dataset.from_generator(count, args=[3], output_types=tf.int32, output_shapes = (), )

In [6]:
a = iter(dataset2)

In [7]:
next(a)

第0次调用……
第1次调用……


<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [8]:
next(a)

第2次调用……


<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [9]:
for i in dataset2:
    print(i)
    print(i.numpy())

第0次调用……
第1次调用……
tf.Tensor(0, shape=(), dtype=int32)
0
tf.Tensor(1, shape=(), dtype=int32)
1
第2次调用……
tf.Tensor(2, shape=(), dtype=int32)
2


**（2）通过接收其他类型的集合类对象创建Dataset对象。**这里所说的集合类型对象包含Python内置的list、tuple，numpy中的ndarray等等。这种创建Dataset对象的方法大多通过from_tensors()和from_tensor_slices()两个方法实现。这两个方法很常用，重点说一说。
- **[from_tensors()](https://tensorflow.google.cn/api_docs/python/tf/data/Dataset#from_tensors)**  
from_tensors()方法接受一个集合类型对象作为参数，返回值为一个TensorDataset类型对象，对象内容、shape因传入参数类型而异。

当接收参数为list或Tensor对象时，返回的情况是一样的，因为TensorFlow内部会将list先转为Tensor对象，然后实例化一个Dataset对象：

In [10]:
a = [0,1,2,3,4]
dataset1 = tf.data.Dataset.from_tensors(a)
dataset1_n = tf.data.Dataset.from_tensors(np.array(a))
dataset1_t = tf.data.Dataset.from_tensors(tf.constant(a))

In [11]:
dataset1,next(iter(dataset1))

(<_TensorDataset element_spec=TensorSpec(shape=(5,), dtype=tf.int32, name=None)>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4])>)

In [12]:
dataset1_n,next(iter(dataset1_n))

(<_TensorDataset element_spec=TensorSpec(shape=(5,), dtype=tf.int32, name=None)>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4])>)

In [13]:
dataset1_t,next(iter(dataset1_t))

(<_TensorDataset element_spec=TensorSpec(shape=(5,), dtype=tf.int32, name=None)>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4])>)

多维结构也是一样的：

In [14]:
a = [0,1,2,3,4]
b = [5,6,7,8,9]
dataset2 = tf.data.Dataset.from_tensors([a,b])
dataset2_n = tf.data.Dataset.from_tensors(np.array([a,b]))
dataset2_t = tf.data.Dataset.from_tensors(tf.constant([a,b]))

In [15]:
dataset2,next(iter(dataset2))

(<_TensorDataset element_spec=TensorSpec(shape=(2, 5), dtype=tf.int32, name=None)>,
 <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
 array([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])>)

In [16]:
dataset2_n,next(iter(dataset2_n))

(<_TensorDataset element_spec=TensorSpec(shape=(2, 5), dtype=tf.int32, name=None)>,
 <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
 array([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])>)

In [17]:
dataset2_t,next(iter(dataset2_t))

(<_TensorDataset element_spec=TensorSpec(shape=(2, 5), dtype=tf.int32, name=None)>,
 <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
 array([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])>)

当接收参数为数组就不一样了，此时Dataset内部内容为一个tuple，tuple的元素是原来tuple元素转换为的Tensor对象：

In [18]:
a = [0,1,2,3,4]
b = [5,6,7,8,9]
dataset3 = tf.data.Dataset.from_tensors((a,b))

In [19]:
for i in dataset3:
    print(type(i))
    print(i)
    for j in i:
        print(j)

<class 'tuple'>
(<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4])>, <tf.Tensor: shape=(5,), dtype=int32, numpy=array([5, 6, 7, 8, 9])>)
tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)
tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int32)


- **[from_tensor_slices()](https://tensorflow.google.cn/api_docs/python/tf/data/Dataset#from_tensor_slices)**  
from_tensor_slices()方法返回一个TensorSliceDataset类对象，TensorSliceDataset对象比from_tensors()方法返回的TensorDataset对象支持更加丰富的操作，例如batch操作等，因此在实际应用中更加广泛。返回的TensorSliceDataset对象内容、shape因传入参数类型而异。

当传入一个list时，时将list中元素逐个转换为Tensor对象然后依次放入Dataset中，所以Dataset中有多个Tensor对象：

In [20]:
a = [0,1,2,3,4]
dataset1 = tf.data.Dataset.from_tensor_slices(a)

In [21]:
dataset1

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [22]:
for i,elem in enumerate(dataset1):
    print(i, '-->', elem)

0 --> tf.Tensor(0, shape=(), dtype=int32)
1 --> tf.Tensor(1, shape=(), dtype=int32)
2 --> tf.Tensor(2, shape=(), dtype=int32)
3 --> tf.Tensor(3, shape=(), dtype=int32)
4 --> tf.Tensor(4, shape=(), dtype=int32)


In [23]:
a = [0,1,2,3,4]
b = [5,6,7,8,9]
dataset2 = tf.data.Dataset.from_tensor_slices([a,b])

In [24]:
dataset2

<_TensorSliceDataset element_spec=TensorSpec(shape=(5,), dtype=tf.int32, name=None)>

In [25]:
for i,elem in enumerate(dataset2):
    print(i, '-->', elem)

0 --> tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)
1 --> tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int32)


当传入参数为tuple时，会将tuple中各元素转换为Tensor对象，然后将第一维度对应位置的切片进行重新组合成一个tuple依次放入到Dataset中，所以在返回的Dataset中有多个tuple。这种形式在对训练集和测试集进行重新组合是非常实用。

In [26]:
a = [0,1,2,3,4]
b = [5,6,7,8,9]
dataset1 = tf.data.Dataset.from_tensor_slices((a,b))

In [27]:
dataset1

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [28]:
for i in dataset1:
    print(i)

(<tf.Tensor: shape=(), dtype=int32, numpy=0>, <tf.Tensor: shape=(), dtype=int32, numpy=5>)
(<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=6>)
(<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=7>)
(<tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=8>)
(<tf.Tensor: shape=(), dtype=int32, numpy=4>, <tf.Tensor: shape=(), dtype=int32, numpy=9>)


In [29]:
c = ['a','b','c','d','e']
dataset3 = tf.data.Dataset.from_tensor_slices((a,b,c))

In [30]:
dataset3

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>

In [31]:
for i in dataset3:
    print(i)

(<tf.Tensor: shape=(), dtype=int32, numpy=0>, <tf.Tensor: shape=(), dtype=int32, numpy=5>, <tf.Tensor: shape=(), dtype=string, numpy=b'a'>)
(<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=6>, <tf.Tensor: shape=(), dtype=string, numpy=b'b'>)
(<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=7>, <tf.Tensor: shape=(), dtype=string, numpy=b'c'>)
(<tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=8>, <tf.Tensor: shape=(), dtype=string, numpy=b'd'>)
(<tf.Tensor: shape=(), dtype=int32, numpy=4>, <tf.Tensor: shape=(), dtype=int32, numpy=9>, <tf.Tensor: shape=(), dtype=string, numpy=b'e'>)


对比总结一下from_generator(）、from_tensor()、from_tensor_slices()这三个方法：
- from_tensors()在形式上与from_tensor_slices()很相似，但其实from_tensors()方法出场频率上比from_tensor_slices()差太多，因为from_tensor_slices()的功能更加符合实际需求，且返回的TensorSliceDataset对象也提供更多的数据处理功能。from_tensors()方法在接受list类型参数时，将整个list转换为Tensor对象放入Dataset中，当接受参数为tuple时，将tuple内元素转换为Tensor对象，然后将这个tuple放入Dataset中。
- from_generator(）方法接受一个可调用的生成器函数作为参数，在遍历Dataset对象时，通过通用生成器函数继续生成新的数据供训练和测试模型使用，这在大数据集合中很实用。
- from_tensor_slices()方法接受参数为list时，将list各元素依次转换为Tensor对象，然后依次放入Dataset中；更为常见的情况是接受的参数为tuple，在这种情况下，要求tuple中各元素第一维度长度必须相等，from_tensor_slices()方法会将tuple各元素第一维度进行拆解，然后将对应位置的元素进行重组成一个个tuple依次放入Dataset中，这一功能在重新组合数据集属性和标签时很有用。另外，from_tensor_slices()方法返回的TensorSliceDataset对象支持batch、shuffle等等功能对数据进一步处理。

**（3）通过读取磁盘中的文件（文本、图片等等）来创建Dataset。**tf.data中提供了TextLineDataset、TFRecordDataset等对象来实现此功能。这部分内容比较多，也比较重要，我打算后续用专门一篇博客来总结这部分内容。

# 2 功能函数

**（1）take()**  

功能：用于返回一个新的Dataset对象，新的Dataset对象包含的数据是原Dataset对象的子集。 

参数：  
- count：整型，用于指定前count条数据用于创建新的Dataset对象，如果count为-1或大于原Dataset对象的size,则用原Dataset对象的全部数据创建新的对象。  

In [32]:
dataset = tf.data.Dataset.range(10)
dataset_take = dataset.take(5)

In [33]:
for i in dataset_take:
    print(i)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)


**（2）batch()**  

功能：将Dataset中连续的数据分割成批。 

参数：  
- batch_size：在单个批次中合并的此数据集的连续元素数。
- drop_remainder：如果最后一批的数据量少于指定的batch_size，是否抛弃最后一批，默认为False，表示不抛弃。

In [34]:
dataset = tf.data.Dataset.range(11)
dataset_batch = dataset.batch(3)

In [35]:
for i in dataset_batch:
    print(i)

tf.Tensor([0 1 2], shape=(3,), dtype=int64)
tf.Tensor([3 4 5], shape=(3,), dtype=int64)
tf.Tensor([6 7 8], shape=(3,), dtype=int64)
tf.Tensor([ 9 10], shape=(2,), dtype=int64)


In [36]:
dataset_batch = dataset.batch(3,drop_remainder=True)

In [37]:
for i in dataset_batch:
    print(i)

tf.Tensor([0 1 2], shape=(3,), dtype=int64)
tf.Tensor([3 4 5], shape=(3,), dtype=int64)
tf.Tensor([6 7 8], shape=(3,), dtype=int64)


In [38]:
train_x = tf.random.uniform((10,3),maxval=100, dtype=tf.int32)
train_y = tf.range(10)

In [39]:
dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))

In [40]:
for i in dataset.take(3):
    print(i)

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([16, 12, 61])>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([62, 20, 33])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1,  1, 31])>, <tf.Tensor: shape=(), dtype=int32, numpy=2>)


In [41]:
dataset_batch = dataset.batch(4)

In [42]:
for i in dataset_batch:
    print(i)

(<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[16, 12, 61],
       [62, 20, 33],
       [ 1,  1, 31],
       [91, 75, 28]])>, <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3])>)
(<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[87, 19, 18],
       [54, 78, 76],
       [43, 36, 53],
       [85, 78, 34]])>, <tf.Tensor: shape=(4,), dtype=int32, numpy=array([4, 5, 6, 7])>)
(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[65,  5, 57],
       [80, 65,  6]])>, <tf.Tensor: shape=(2,), dtype=int32, numpy=array([8, 9])>)


为什么在训练模型时要将Dataset分割成一个个batch呢？
- 对于小数据集是否使用batch关系不大，但是对于大数据集如果不分割成batch意味着将这个数据集一次性输入模型中，容易造成内存爆炸。
- 通过并行化提高内存的利用率。就是尽量让你的GPU满载运行，提高训练速度。
- 单个epoch的迭代次数减少了，参数的调整也慢了，假如要达到相同的识别精度，需要更多的epoch。
- 适当Batch Size使得梯度下降方向更加准确。

**（3）padded_batch()**  

功能： batch()的进阶版，可以对shape不一致的连续元素进行分批。  

参数：  
- batch_size：在单个批次中合并的此数据集的连续元素个数。
- padded_shapes：tf.TensorShape或其他描述tf.int64矢量张量对象，表示在批处理之前每个输入元素的各个组件应填充到的形状。如果参数中有None，则表示将填充为每个批次中该尺寸的最大尺寸。
- padding_values：要用于各个组件的填充值。默认值0用于数字类型，字符串类型则默认为空字符。
- drop_remainder：如果最后一批的数据量少于指定的batch_size，是否抛弃最后一批，默认为False，表示不抛弃。

In [43]:
dataset = tf.data.Dataset.range(10)

In [44]:
dataset = dataset.map(lambda x: tf.fill([tf.cast(x, tf.int32)], x))

In [45]:
dataset_padded = dataset.padded_batch(4, padded_shapes=(None,))

In [46]:
for batch in dataset_padded:
    print(batch.numpy())
    print('---------------------')

[[0 0 0]
 [1 0 0]
 [2 2 0]
 [3 3 3]]
---------------------
[[4 4 4 4 0 0 0]
 [5 5 5 5 5 0 0]
 [6 6 6 6 6 6 0]
 [7 7 7 7 7 7 7]]
---------------------
[[8 8 8 8 8 8 8 8 0]
 [9 9 9 9 9 9 9 9 9]]
---------------------


In [47]:
dataset_padded = dataset.padded_batch(4, padded_shapes=(10,),padding_values=tf.constant(9,dtype=tf.int64))  # 修改填充形状和填充元素

In [48]:
for batch in dataset_padded:
    print(batch.numpy())
    print('---------------------')

[[9 9 9 9 9 9 9 9 9 9]
 [1 9 9 9 9 9 9 9 9 9]
 [2 2 9 9 9 9 9 9 9 9]
 [3 3 3 9 9 9 9 9 9 9]]
---------------------
[[4 4 4 4 9 9 9 9 9 9]
 [5 5 5 5 5 9 9 9 9 9]
 [6 6 6 6 6 6 9 9 9 9]
 [7 7 7 7 7 7 7 9 9 9]]
---------------------
[[8 8 8 8 8 8 8 8 9 9]
 [9 9 9 9 9 9 9 9 9 9]]
---------------------


**（4）map()**  

功能： 以dataset中每一位元素为参数执行pap_func()方法，这一功能在数据预处理中修改dataset中元素是很实用。

参数：
- map_func:回调方法。

In [49]:
def change_dtype(t):  # 将类型修改为int32
    return tf.cast(t,dtype=tf.int32)

In [50]:
dataset = tf.data.Dataset.range(3)

In [51]:
for i in dataset:
    print(i)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)


In [52]:
dataset_map = dataset.map(change_dtype)

In [53]:
for i in dataset_map:
    print(i)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)


map_func的参数必须对应dataset中的元素类型，例如，如果dataset中元素是tuple，map_func可以这么定义：

In [54]:
def change_dtype_2(t1,t2):
    return t1/10,tf.cast(t2,dtype=tf.int32)*(-1)  # 第一位元素除以10，第二为元素乘以-1

In [55]:
dataset = tf.data.Dataset.from_tensor_slices((tf.range(3),tf.range(3)))

In [56]:
dataset_map = dataset.map(change_dtype_2)

In [57]:
for i in dataset_map:
    print(i)

(<tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=float64, numpy=0.1>, <tf.Tensor: shape=(), dtype=int32, numpy=-1>)
(<tf.Tensor: shape=(), dtype=float64, numpy=0.2>, <tf.Tensor: shape=(), dtype=int32, numpy=-2>)


**（5）filter()**  

功能：对Dataset中每一个执行指定过滤方法进行过滤，返回过滤后的Dataset对象  

参数：
- predicate：过滤方法，返回值必须为True或False

In [58]:
dataset = tf.data.Dataset.range(5)

In [59]:
def filter_func(t):  # 过滤出值为偶数的元素
    if t % 2 == 0:
        return True
    else:
        return False

In [60]:
dataset_filter = dataset.filter(filter_func)

In [61]:
for i in dataset_filter:
    print(i)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)


**（6）shuffle()**

功能：随机打乱数据  

参数：
- buffer_size：缓冲区大小，姑且认为是混乱程度吧，当值为1时，完全不打乱，当值为整个Dataset元素总数时，完全打乱。
- seed：将用于创建分布的随机种子。
- reshuffle_each_iteration：如果为true，则表示每次迭代数据集时都应进行伪随机重排，默认为True。

In [62]:
dataset = tf.data.Dataset.range(5)

In [63]:
dataset_s = dataset.shuffle(1)

In [64]:
for i in dataset_s:
    print(i)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)


In [65]:
dataset_s = dataset.shuffle(5)

In [66]:
for i in dataset_s:
    print(i)

tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)


**（7）repeat()**  

功能：对Dataset中的数据进行重复，以创建新的Dataset

参数：
- count：重复次数，默认为None，表示不重复，当值为-1时，表示无限重复。

In [67]:
dataset = tf.data.Dataset.range(3)

In [68]:
dataset_repeat = dataset.repeat(3)

In [69]:
for i in dataset_repeat:
    print(i)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)


In [70]:
dataset = tf.data.Dataset.range(5).map(lambda x: ([x, x, x],[1, 0, 0]))
# train_y = tf.random.uniform((100,3),maxval=2, dtype=tf.int32)
# print(dataset)

In [71]:
# for i in dataset.take(1):
#     print(i)

In [116]:
def windowed_dataset(dataset: tf.data.Dataset,
                     window_size=5,
                     shift=1,
                     stride=1):
    windows = dataset.window(window_size,
                             shift=shift,
                             stride=stride,
                             drop_remainder=True)

    def sub_to_batch(sub):
        # 如果特征和标签在同一张量中，需先分离
        if sub is not None and isinstance(sub, tuple):
            features, labels = sub
        else:
            features = sub  # 假设sub本身就是特征
            labels = None  # 或者相应地获取标签

        features_batches = features.batch(window_size, drop_remainder=True)

        # 如果有多个标签，也对它们进行批处理
        if labels is not None:
            labels_batches = labels.batch(window_size, drop_remainder=True)

            # 返回特征和标签的批处理结果作为元组
            return tf.data.Dataset.zip((features_batches, labels_batches))
        else:
            return features_batches

    windows = windows.flat_map(sub_to_batch)
    return windows

In [120]:
import tensorflow as tf
import pandas as pd

# 假设有一个包含多个特征和多标签的DataFrame
df = pd.DataFrame({
    'feature1': np.arange(0, 10),  # 第一个特征列
    'feature2': np.arange(10, 20),  # 第二个特征列
    'label1': np.arange(10, 20),  # 第一个标签列
    'label2': np.arange(0, 10),  # 第二个标签列
    # 更多特征和标签...
})
# print(df['label1'])
# 定义特征列名和标签列名
feature_col_names = ['feature1', 'feature2']
label_col_names = ['label1', 'label2']

# 将DataFrame转换为NumPy数组
features = df[feature_col_names].values
labels = df[label_col_names].values

dataset_features = tf.data.Dataset.from_tensor_slices(features)
dataset_labels = tf.data.Dataset.from_tensor_slices(labels)
# 创建数据集
dataset = tf.data.Dataset.zip((dataset_features, dataset_labels))

# 如果需要对特征或标签做预处理（如归一化、标准化等），可以添加map操作
# dataset = dataset.map(lambda feat, lab: (preprocess_features(feat), preprocess_labels(lab)))

# 对于时间序列数据或其他需要窗口滑动的情况，可以使用window方法并应用batch
# window_size 表示每个窗口包含的样本数
window_size = 2
shift = 1
stride = 1


def windowed_dataset(ds: tf.data.Dataset,
                     window_size=window_size,
                     shift=shift,
                     stride=stride):
    windows = ds.window(window_size,
                        shift=shift,
                        stride=stride,
                        drop_remainder=True,name='op_window')

    def sub_to_batch(sub):
        return sub.batch(window_size, drop_remainder=True,name='op_sub_to batch')

    windows = windows.flat_map(sub_to_batch)
    return windows


# 应用窗口滑动
dataset = windowed_dataset(dataset)

# 设置批处理大小
batch_size = 32
# dataset = dataset.batch(batch_size)

# 验证数据集形状
for feat, lab in dataset:
    print('Features shape:',
          feat.shape)  # (batch_size, window_size, num_features)
    print('Labels shape:', lab.shape)  # (batch_size, window_size, num_labels)

TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__sub_to_batch() takes 1 positional argument but 2 were given
